In [2]:
# importação de bibliotecas
import pandas as pd
import io
import os
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import time
from datetime import datetime
from bs4 import BeautifulSoup
import random

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bestcaptchasolverapi3.bestcaptchasolverapi import BestCaptchaSolverAPI
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
import requests
import pyautogui
from selenium.common.exceptions import TimeoutException


from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pyautogui
from PIL import Image
import time
import os

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

import os
import zipfile

In [24]:
def initialize_chrome_driver(estados):
    # Configurações do Chrome
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")  # Adiciona a opção de iniciar maximizado
    prefs = {'download.default_directory' : f'C:\\CL2022\\Flash_diagnostcAPP\\selenium\\SICAR_ARQUIVOS\\{estados}',
            "download.prompt_for_download": False,
            "directory_upgrade": True,
            "safebrowsing.enabled": True
            }
    chrome_options.add_experimental_option('prefs', prefs)

    # Inicializa o WebDriver com as opções configuradas
    driver = webdriver.Chrome(options=chrome_options)

    return driver

def navigate_to_url(driver, url):
    driver.get(url)
    time.sleep(4)  # Aguarda para garantir que a página seja carregada completamente
    driver.get(url)  # Recarrega a página, se necessário


#Para entrar no estado
def click_download_button(driver, state):
    try:
        botao_download = driver.find_element(By.XPATH, f"//button[@data-estado='{state}']")
        botao_download.click()
        time.sleep(2)  # Tempo para aguardar a resposta do clique
    except Exception as e:
        print(f"Ocorreu um erro ao clicar no botão: {e}")


def capture_captcha(driver):
    try:
        # Esperar até que a imagem do CAPTCHA esteja visível
        wait = WebDriverWait(driver, 10)
        wait.until(EC.visibility_of_element_located((By.ID, "img-ReCaptcha-base-downloads")))

        # Captura de tela com PyAutoGUI
        screenshot_path = 'C:/CL2022/Flash_diagnostcAPP/selenium/selenium_sicar/screenshot.png'  # Caminho fixo para salvar a captura de tela
        screenshot = pyautogui.screenshot()
        screenshot.save(screenshot_path)

        time.sleep(2)  # Pausa para garantir que a captura de tela ocorra

        crop_captcha_image(screenshot_path) #função para salvar clipado
    except Exception as e:
        print(f"Ocorreu um erro durante a captura do CAPTCHA: {e}")

def crop_captcha_image(screenshot_path):

    cropped_captcha_path = 'C:/CL2022/Flash_diagnostcAPP/selenium/selenium_sicar/cropped_captcha.png'
    left, top, right, bottom = 830, 315, 1020, 380

    try:
        # Abrir a imagem original
        img = Image.open(screenshot_path)

        # Recortar a imagem com as coordenadas especificadas
        captcha_img = img.crop((left, top, right, bottom))

        # Salvar a imagem recortada
        captcha_img.save(cropped_captcha_path)

        time.sleep(2)  # Pausa para garantir a conclusão da operação de arquivo
    except Exception as e:
        print(f"Ocorreu um erro ao processar a imagem: {e}")

def resolve_captcha(cropped_captcha_path = 'C:/CL2022/Flash_diagnostcAPP/selenium/selenium_sicar/cropped_captcha.png'):
    # Token de acesso e caminho da imagem fixos
    access_token = '3E7FD47B753B4481A4D409E873FCC7DA'
    image_path = cropped_captcha_path

    # Configurar os dados para a resolução do CAPTCHA
    data = {
        'image': image_path,
        'is_case': True,
        'is_phrase': False,
        'is_math': False,
        'alphanumeric': 1,  # Use 1 para alfanumérico ou 2 para apenas numérico
        'minlength': 4,
        'maxlength': 5
    }

    try:
        # Inicializar o serviço de resolução de CAPTCHA
        bcs = BestCaptchaSolverAPI(access_token)

        # Enviar a imagem do CAPTCHA para resolução
        captcha_id = bcs.submit_image_captcha(data)
        time.sleep(10)  # Esperar um tempo para a resolução do CAPTCHA

        # Obter o texto resolvido do CAPTCHA
        image_text = bcs.retrieve(captcha_id)['text']
        time.sleep(1)
        submit_captcha(driver, captcha_text = image_text)
    except Exception as e:
        print(f"Ocorreu um erro ao resolver o CAPTCHA: {e}")
        return None

def submit_captcha(driver, captcha_text):
    try:

        erroBaixar = True

        wait = WebDriverWait(driver, 10)

        # Encontrar o campo de entrada e inserir o texto do CAPTCHA
        input_element = wait.until(EC.element_to_be_clickable((By.ID, 'form-ReCaptcha-download')))
        input_element.clear()        
        input_element.send_keys(captcha_text + Keys.TAB)

        # Encontrar e clicar no botão de confirmação
        confirm_button = wait.until(EC.element_to_be_clickable((By.ID, 'btn-confirmar-download-base-poligono')))
        time.sleep(1)
        confirm_button.click()
    except TimeoutException:
        erroBaixar = False

    # Verificar se ocorreu um erro
    if erroBaixar:
        check_error_and_refresh_captcha(driver)

    else:
        print("Download iniciado com sucesso.")

#Em caso de erros ao fazer o download por causa do CAPTCHA
def check_error_and_refresh_captcha(driver):
    try:
        wait = WebDriverWait(driver, 5)

        # Verificar se a mensagem de erro está visível
        wait.until(EC.visibility_of_element_located((By.XPATH, "//*[@id='alert-download-error']")))

        # Se a mensagem de erro for encontrada, atualizar o CAPTCHA
        botaoMudarCaptcha = wait.until(EC.element_to_be_clickable((By.ID, 'btn-atualizar-ReCaptcha')))
        botaoMudarCaptcha.click()

        time.sleep(10) # Esse tempo ser para atualizar a página e deixar os elementos do mesmo tamanho

        capture_captcha(driver)

        resolve_captcha()

        return True  # Indica que foi encontrado um erro e o CAPTCHA foi atualizado
    except Exception as e:
        return False  # Nenhum erro encontrado ou outro problema ocorreu



#Perímetros dos imóveis
def Perimetro_shp_imovel(driver):
    try:
        # Espera até que o botão específico esteja clicável
        wait = WebDriverWait(driver, 10)  # Ajuste o tempo de espera conforme necessário
        download_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@data-tipodownload='AREA_IMOVEL']")))

        # Clica no botão de download para "Perímetros dos imóveis"
        download_button.click()
        # Tempo para aguardar ações subsequentes, como a apresentação de um CAPTCHA
        time.sleep(5)

        #função para capturar captcha
        capture_captcha(driver)

        #função para resolver e baixar dados
        resolve_captcha()

    except Exception as e:
        print(f"Ocorreu um erro durante o download do SHP do imóvel: {e}")


#Perímetros das Área de Preservação Permanente
def Perimetro_shp_apps(driver):
    try:
        # Espera até que o botão específico esteja clicável
        wait = WebDriverWait(driver, 10)  # Ajuste o tempo de espera conforme necessário
        download_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@data-tipodownload='APPS']")))

        # Clica no botão de download para "Perímetros dos imóveis"
        download_button.click()

        # Tempo para aguardar ações subsequentes, como a apresentação de um CAPTCHA
        time.sleep(5)

        #função para capturar captcha
        capture_captcha(driver)

        #função para resolver e baixar dados
        resolve_captcha()
        time.sleep(40)
    except Exception as e:
        print(f"Ocorreu um erro durante o download do SHP das apps: {e}")


#Perímetros da Remanescente de Vegetação Nativa
def Perimetro_shp_Remanescente_nativa(driver):
    try:
        # Espera até que o botão específico esteja clicável
        wait = WebDriverWait(driver, 10)  # Ajuste o tempo de espera conforme necessário
        download_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@data-tipodownload='VEGETACAO_NATIVA']")))

        # Clica no botão de download para "Perímetros dos imóveis"
        download_button.click()

        # Tempo para aguardar ações subsequentes, como a apresentação de um CAPTCHA
        time.sleep(30)

        #função para capturar captcha
        capture_captcha(driver)

        #função para resolver e baixar dados
        resolve_captcha()

        time.sleep(60)

    except Exception as e:
        print(f"Ocorreu um erro durante o download do SHP das áreas de remanescentes: {e}")


#Área Consolidada
def Perimetro_shp_area_consolidada(driver):
    try:
        # Espera até que o botão específico esteja clicável
        wait = WebDriverWait(driver, 10)  # Ajuste o tempo de espera conforme necessário
        download_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@data-tipodownload='AREA_CONSOLIDADA']")))

        # Clica no botão de download para "Perímetros dos imóveis"
        download_button.click()

        # Tempo para aguardar ações subsequentes, como a apresentação de um CAPTCHA
        time.sleep(5)

        #função para capturar captcha
        capture_captcha(driver)

        #função para resolver e baixar dados
        resolve_captcha()

    except Exception as e:
        print(f"Ocorreu um erro durante o download do SHP da Área Consolidada: {e}")


#Perímetros da área de pousio
def Perimetro_shp_area_pousio(driver):
    try:
        # Espera até que o botão específico esteja clicável
        wait = WebDriverWait(driver, 10)  # Ajuste o tempo de espera conforme necessário
        download_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@data-tipodownload='AREA_POUSIO']")))

        # Clica no botão de download para "Perímetros dos imóveis"
        download_button.click()

        # Tempo para aguardar ações subsequentes, como a apresentação de um CAPTCHA
        time.sleep(5)

        #função para capturar captcha
        capture_captcha(driver)

        #função para resolver e baixar dados
        resolve_captcha()

    except Exception as e:
        print(f"Ocorreu um erro durante o download do SHP das áreas de remanescentes: {e}")


#Perímetros da Hidrografia
def Perimetro_shp_hidrografia(driver):
    try:
        # Espera até que o botão específico esteja clicável
        wait = WebDriverWait(driver, 10)  # Ajuste o tempo de espera conforme necessário
        download_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@data-tipodownload='HIDROGRAFIA']")))

        # Clica no botão de download para "Perímetros dos imóveis"
        download_button.click()

        # Tempo para aguardar ações subsequentes, como a apresentação de um CAPTCHA
        time.sleep(5)

        #função para capturar captcha
        capture_captcha(driver)

        #função para resolver e baixar dados
        resolve_captcha()

    except Exception as e:
        print(f"Ocorreu um erro durante o download do SHP das áreas de remanescentes: {e}")



#Perímetros Uso Restrito
def Perimetro_shp_uso_restrito(driver):
    try:
        # Espera até que o botão específico esteja clicável
        wait = WebDriverWait(driver, 10)  # Ajuste o tempo de espera conforme necessário
        download_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@data-tipodownload='USO_RESTRITO']")))

        # Clica no botão de download para "Perímetros dos imóveis"
        download_button.click()

        # Tempo para aguardar ações subsequentes, como a apresentação de um CAPTCHA
        time.sleep(5)

        #função para capturar captcha
        capture_captcha(driver)

        #função para resolver e baixar dados
        resolve_captcha()

    except Exception as e:
        print(f"Ocorreu um erro durante o download do SHP das áreas de remanescentes: {e}")


#Perímetros Servidão Administrativa
def Perimetro_shp_servidao_administrativa(driver):
    try:
        # Espera até que o botão específico esteja clicável
        wait = WebDriverWait(driver, 10)  # Ajuste o tempo de espera conforme necessário
        download_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@data-tipodownload='SERVIDAO_ADMINISTRATIVA']")))

        # Clica no botão de download para "Perímetros dos imóveis"
        download_button.click()

        # Tempo para aguardar ações subsequentes, como a apresentação de um CAPTCHA
        time.sleep(5)

        #função para capturar captcha
        capture_captcha(driver)

        #função para resolver e baixar dados
        resolve_captcha()

    except Exception as e:
        print(f"Ocorreu um erro durante o download do SHP das áreas de remanescentes: {e}")


#Perímetros RESERVA LEGAL
def Perimetro_shp_reserva_legal(driver):
    try:
        # Espera até que o botão específico esteja clicável
        wait = WebDriverWait(driver, 10)  # Ajuste o tempo de espera conforme necessário
        download_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@data-tipodownload='RESERVA_LEGAL']")))

        # Clica no botão de download para "Perímetros dos imóveis"
        download_button.click()

        # Tempo para aguardar ações subsequentes, como a apresentação de um CAPTCHA
        time.sleep(5)

        #função para capturar captcha
        capture_captcha(driver)

        #função para resolver e baixar dados
        resolve_captcha()

    except Exception as e:
        print(f"Ocorreu um erro durante o download do SHP das áreas de remanescentes: {e}")



def extrair_zip(estado):
    directory = f'C:\\CL2022\\Flash_diagnostcAPP\\selenium\\SICAR_ARQUIVOS\\{estado}'
    
    for item in os.listdir(directory):
        time.sleep(5)
        if item.endswith('.zip'):
            file_name = os.path.join(directory, item)
            folder_name = os.path.splitext(item)[0]  # Remove a extensão .zip para criar o nome da pasta
            full_folder_path = os.path.join(directory, folder_name)
            
            if not os.path.exists(full_folder_path):  # Cria a pasta se ela não existir
                os.makedirs(full_folder_path)
            
            # Extrai o arquivo ZIP na sua respectiva pasta
            with zipfile.ZipFile(file_name, 'r') as zip_ref:
                zip_ref.extractall(full_folder_path)
            
            time.sleep(20)
            
            os.remove(file_name)  # Remove o arquivo ZIP após a extração

    print("Todos os arquivos ZIP foram extraídos para suas respectivas pastas e os arquivos ZIP foram deletados.")


# função para esperar o download dos arquivos
def esperar_arquivos(directory, totalArquivos = 1):
    while True:
        # Contar o número de arquivos no diretório
        numero_de_arquivos = len(os.listdir(directory))

        # Se houver mais de um arquivo, execute a função e saia do loop
        if numero_de_arquivos >= totalArquivos:
            # Esperar um pouco antes de verificar novamente

            time.sleep(5)
            break



estados = ['GO', 'MT', 'MS', 'DF']



for estado in estados:
    print(f'Iniciando...{estado}')
    driver = initialize_chrome_driver(estados = estado)
    url = "https://www.car.gov.br/publico/estados/downloads"

    #Diretorio dos arquivos zipados
    directory = f'C:\\CL2022\\Flash_diagnostcAPP\\selenium\\SICAR_ARQUIVOS\\{estado}'
    navigate_to_url(driver, url)

    #Para entrar no estado
    click_download_button(driver, estado)

    #Perímetros dos imóveis
    Perimetro_shp_imovel(driver)
    esperar_arquivos(directory, totalArquivos = 1)

    # #Perímetros das apps
    Perimetro_shp_apps(driver)
    esperar_arquivos(directory, totalArquivos = 2)

    ##Perímetros da Remanescente de Vegetação Nativa
    Perimetro_shp_Remanescente_nativa(driver)
    esperar_arquivos(directory, totalArquivos = 3)

    #Área Consolidada
    Perimetro_shp_area_consolidada(driver)
    esperar_arquivos(directory, totalArquivos = 4)

    #Perímetros da área de pousio
    Perimetro_shp_area_pousio(driver)
    esperar_arquivos(directory, totalArquivos = 5)


    #Perímetros da Hidrografia
    Perimetro_shp_hidrografia(driver)
    esperar_arquivos(directory, totalArquivos = 6)

    #Perímetros Uso Restrito
    Perimetro_shp_uso_restrito(driver)
    esperar_arquivos(directory, totalArquivos = 7)

    #Perímetros Servidão Administrativa
    Perimetro_shp_servidao_administrativa(driver)
    esperar_arquivos(directory, totalArquivos = 8)

    #Perímetros RESERVA LEGAL
    Perimetro_shp_reserva_legal(driver)
    esperar_arquivos(directory, totalArquivos = 9
    
    )
    time.sleep(10)
    extrair_zip(estado=estado)


Iniciando...GO
Todos os arquivos ZIP foram extraídos para suas respectivas pastas e os arquivos ZIP foram deletados.
Iniciando...MT
Ocorreu um erro ao resolver o CAPTCHA: ('Connection aborted.', ConnectionResetError(10054, 'Foi forçado o cancelamento de uma conexão existente pelo host remoto', None, 10054, None))
Todos os arquivos ZIP foram extraídos para suas respectivas pastas e os arquivos ZIP foram deletados.
Iniciando...MS
Todos os arquivos ZIP foram extraídos para suas respectivas pastas e os arquivos ZIP foram deletados.
Iniciando...DF
Download iniciado com sucesso.
Download iniciado com sucesso.
Todos os arquivos ZIP foram extraídos para suas respectivas pastas e os arquivos ZIP foram deletados.
